# Photo Quality Classifier — Stage 1: Good / Bad / Unreadable

**Goal:** Train a 3-class classifier for data sticker photos:
- **Good** — clear, legible, usable
- **Bad** — poor photo quality (blurry, dark, angled) — tech should retake
- **Unreadable** — physical label is damaged/faded beyond recovery — retake won't help

**Model:** MobileNetV2 with transfer learning (pre-trained on ImageNet).
Lightweight, fast inference, easy to deploy later.

**Workflow:**
1. Upload zipped `good/`, `bad/`, and `unreadable/` folders
2. Data augmentation to increase training variety
3. Train with frozen base → fine-tune top layers
4. Evaluate with accuracy, precision, recall, F1, confusion matrix
5. Export as `.h5` and TFLite

---

**Run in Google Colab with GPU enabled:**
Runtime → Change runtime type → GPU

## 1. Setup & Upload Data

In [ ]:
import os
import zipfile
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay

print(f"TensorFlow version: {tf.__version__}")
print(f"GPU available: {len(tf.config.list_physical_devices('GPU')) > 0}")

In [ ]:
# Upload your training data zip file.
# Expected structure inside the zip:
#   sorted/good/*.jpg
#   sorted/bad/*.jpg
#   sorted/unreadable/*.jpg

from google.colab import files

print("Upload your training data zip (sorted.zip containing good/, bad/, and unreadable/ folders):")
uploaded = files.upload()

zip_filename = list(uploaded.keys())[0]
print(f"Uploaded: {zip_filename}")

In [ ]:
# Extract the zip
with zipfile.ZipFile(zip_filename, 'r') as z:
    z.extractall('training_data')

# Auto-detect the data directory structure
# Supports: training_data/good & training_data/bad & training_data/unreadable
#       or: training_data/sorted/good & training_data/sorted/bad & training_data/sorted/unreadable
if os.path.isdir('training_data/sorted/good'):
    DATA_DIR = 'training_data/sorted'
elif os.path.isdir('training_data/good'):
    DATA_DIR = 'training_data'
else:
    # List what was extracted to help debug
    for root, dirs, _files in os.walk('training_data'):
        for d in dirs:
            print(os.path.join(root, d))
    raise FileNotFoundError("Could not find good/, bad/, unreadable/ folders. Check zip structure.")

good_count = len(os.listdir(os.path.join(DATA_DIR, 'good')))
bad_count = len(os.listdir(os.path.join(DATA_DIR, 'bad')))
unreadable_count = len(os.listdir(os.path.join(DATA_DIR, 'unreadable')))
print(f"Data directory: {DATA_DIR}")
print(f"Good photos:       {good_count}")
print(f"Bad photos:        {bad_count}")
print(f"Unreadable photos: {unreadable_count}")
print(f"Total:             {good_count + bad_count + unreadable_count}")

## 2. Data Loading & Augmentation

In [ ]:
# Configuration
IMG_SIZE = 224        # MobileNetV2 default input size
BATCH_SIZE = 32
VALIDATION_SPLIT = 0.2
SEED = 42

In [ ]:
# Load training set (3 classes: bad=0, good=1, unreadable=2 — alphabetical)
train_ds = keras.utils.image_dataset_from_directory(
    DATA_DIR,
    validation_split=VALIDATION_SPLIT,
    subset='training',
    seed=SEED,
    image_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    label_mode='categorical',
)

# Load validation set
val_ds = keras.utils.image_dataset_from_directory(
    DATA_DIR,
    validation_split=VALIDATION_SPLIT,
    subset='validation',
    seed=SEED,
    image_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    label_mode='categorical',
)

class_names = train_ds.class_names
NUM_CLASSES = len(class_names)
print(f"Classes ({NUM_CLASSES}): {class_names}")
print(f"Class mapping: " + ", ".join(f"{name}={i}" for i, name in enumerate(class_names)))

In [ ]:
# Preview a batch of training images
plt.figure(figsize=(12, 8))
for images, labels in train_ds.take(1):
    for i in range(min(12, len(images))):
        ax = plt.subplot(3, 4, i + 1)
        plt.imshow(images[i].numpy().astype('uint8'))
        label_idx = np.argmax(labels[i].numpy())
        plt.title(class_names[label_idx])
        plt.axis('off')
plt.suptitle('Sample Training Images', fontsize=14)
plt.tight_layout()
plt.show()

In [ ]:
# Data augmentation layer (applied only during training)
data_augmentation = keras.Sequential([
    layers.RandomFlip('horizontal'),
    layers.RandomRotation(0.15),
    layers.RandomZoom(0.1),
    layers.RandomBrightness(0.2),
    layers.RandomContrast(0.2),
], name='data_augmentation')

# Preview augmented images
plt.figure(figsize=(12, 4))
for images, _ in train_ds.take(1):
    original = images[0]
    for i in range(6):
        ax = plt.subplot(1, 6, i + 1)
        augmented = data_augmentation(tf.expand_dims(original, 0))
        plt.imshow(augmented[0].numpy().astype('uint8'))
        plt.axis('off')
        plt.title('aug' if i > 0 else 'original')
plt.suptitle('Augmentation Preview', fontsize=14)
plt.tight_layout()
plt.show()

In [ ]:
# Performance optimization: prefetch data
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

## 3. Build Model (MobileNetV2 + Transfer Learning)

In [ ]:
# Load pre-trained MobileNetV2 (without top classification layer)
base_model = keras.applications.MobileNetV2(
    input_shape=(IMG_SIZE, IMG_SIZE, 3),
    include_top=False,
    weights='imagenet',
)

# Freeze the base model — we'll only train the new head initially
base_model.trainable = False

print(f"Base model layers: {len(base_model.layers)}")
print(f"Base model params: {base_model.count_params():,}")

In [ ]:
# Build the full model
inputs = keras.Input(shape=(IMG_SIZE, IMG_SIZE, 3))

# Augmentation (only active during training)
x = data_augmentation(inputs)

# MobileNetV2 preprocessing (scales pixels to [-1, 1])
x = keras.applications.mobilenet_v2.preprocess_input(x)

# Base model (frozen)
x = base_model(x, training=False)

# Classification head — 3 classes with softmax
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.3)(x)
x = layers.Dense(128, activation='relu')(x)
x = layers.Dropout(0.2)(x)
outputs = layers.Dense(NUM_CLASSES, activation='softmax')(x)

model = keras.Model(inputs, outputs, name='quality_classifier')
model.summary()

## 4. Train — Phase 1: Frozen Base

In [ ]:
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-3),
    loss='categorical_crossentropy',
    metrics=['accuracy'],
)

early_stop = keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=3,
    restore_best_weights=True,
)

print("Phase 1: Training classification head (base frozen)...")
history1 = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10,
    callbacks=[early_stop],
)

## 5. Train — Phase 2: Fine-Tune Top Layers

In [ ]:
# Unfreeze the top ~30% of the base model for fine-tuning
base_model.trainable = True
fine_tune_from = 100  # MobileNetV2 has 154 layers — freeze first 100

for layer in base_model.layers[:fine_tune_from]:
    layer.trainable = False

trainable_count = sum(1 for l in base_model.layers if l.trainable)
print(f"Fine-tuning {trainable_count} of {len(base_model.layers)} base layers")

# Use a lower learning rate for fine-tuning to avoid catastrophic forgetting
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-4),
    loss='categorical_crossentropy',
    metrics=['accuracy'],
)

early_stop_ft = keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True,
)

print("Phase 2: Fine-tuning top layers...")
history2 = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=20,
    callbacks=[early_stop_ft],
)

## 6. Training History

In [ ]:
# Combine training histories from both phases
def combine_histories(h1, h2):
    combined = {}
    for key in h1.history:
        combined[key] = h1.history[key] + h2.history[key]
    return combined

history = combine_histories(history1, history2)
phase1_epochs = len(history1.history['loss'])

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Accuracy
ax1.plot(history['accuracy'], label='Train')
ax1.plot(history['val_accuracy'], label='Validation')
ax1.axvline(x=phase1_epochs - 0.5, color='gray', linestyle='--', alpha=0.5, label='Fine-tune start')
ax1.set_title('Model Accuracy')
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Accuracy')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Loss
ax2.plot(history['loss'], label='Train')
ax2.plot(history['val_loss'], label='Validation')
ax2.axvline(x=phase1_epochs - 0.5, color='gray', linestyle='--', alpha=0.5, label='Fine-tune start')
ax2.set_title('Model Loss')
ax2.set_xlabel('Epoch')
ax2.set_ylabel('Loss')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.suptitle('Training History', fontsize=14)
plt.tight_layout()
plt.show()

# Print final metrics
print(f"\nFinal training accuracy:   {history['accuracy'][-1]:.4f}")
print(f"Final validation accuracy: {history['val_accuracy'][-1]:.4f}")

## 7. Evaluation

In [ ]:
# Collect all validation predictions
y_true = []
y_pred_list = []

for images, labels in val_ds:
    preds = model.predict(images, verbose=0)
    y_true.extend(np.argmax(labels.numpy(), axis=1))
    y_pred_list.extend(np.argmax(preds, axis=1))

y_true = np.array(y_true)
y_pred = np.array(y_pred_list)

# Classification report
print("Classification Report:")
print("=" * 55)
print(classification_report(y_true, y_pred, target_names=class_names))

In [ ]:
# Confusion matrix
cm = confusion_matrix(y_true, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=class_names)

fig, ax = plt.subplots(figsize=(7, 7))
disp.plot(ax=ax, cmap='Blues', values_format='d')
plt.title('Confusion Matrix — Validation Set')
plt.tight_layout()
plt.show()

# Summary stats
total = len(y_true)
correct = (y_pred == y_true).sum()
print(f"\nValidation: {correct}/{total} correct ({correct/total*100:.1f}%)")

In [ ]:
# Sample predictions on validation images
plt.figure(figsize=(16, 8))
shown = 0
for images, labels in val_ds:
    preds = model.predict(images, verbose=0)
    for i in range(len(images)):
        if shown >= 16:
            break
        ax = plt.subplot(4, 4, shown + 1)
        plt.imshow(images[i].numpy().astype('uint8'))
        true_idx = np.argmax(labels[i].numpy())
        pred_idx = np.argmax(preds[i])
        confidence = preds[i][pred_idx]
        true_label = class_names[true_idx]
        pred_label = class_names[pred_idx]
        color = 'green' if true_idx == pred_idx else 'red'
        plt.title(f'T:{true_label} P:{pred_label}\n{confidence:.0%}', color=color, fontsize=9)
        plt.axis('off')
        shown += 1
    if shown >= 16:
        break

plt.suptitle('Sample Predictions (green=correct, red=wrong)', fontsize=14)
plt.tight_layout()
plt.show()

## 8. Export Model

In [ ]:
# Save as .h5 (Keras format)
h5_path = 'quality_classifier.h5'
model.save(h5_path)
h5_size = os.path.getsize(h5_path) / (1024 * 1024)
print(f"Saved Keras model: {h5_path} ({h5_size:.1f} MB)")

# Save as SavedModel format (for TFLite conversion)
saved_model_dir = 'quality_classifier_saved_model'
model.save(saved_model_dir)
print(f"Saved TF SavedModel: {saved_model_dir}/")

In [ ]:
# Convert to TFLite for lightweight deployment
converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()

tflite_path = 'quality_classifier.tflite'
with open(tflite_path, 'wb') as f:
    f.write(tflite_model)

tflite_size = os.path.getsize(tflite_path) / (1024 * 1024)
print(f"Saved TFLite model: {tflite_path} ({tflite_size:.1f} MB)")
print(f"Size reduction: {h5_size/tflite_size:.1f}x smaller than .h5")

In [ ]:
# Download the trained models
from google.colab import files

print("Downloading models...")
files.download(h5_path)
files.download(tflite_path)

## 9. Quick Inference Test

Use this cell to test the model on individual images after training.

In [ ]:
def predict_quality(image_path, model, img_size=224):
    """Predict whether a single photo is good, bad, or unreadable."""
    img = keras.utils.load_img(image_path, target_size=(img_size, img_size))
    img_array = keras.utils.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)

    predictions = model.predict(img_array, verbose=0)[0]
    pred_idx = np.argmax(predictions)
    label = class_names[pred_idx]
    confidence = predictions[pred_idx]

    return label, float(confidence)

# Example: test on a few validation images
import random
test_dir = os.path.join(DATA_DIR, 'good')
test_files = random.sample(os.listdir(test_dir), min(3, len(os.listdir(test_dir))))
for fname in test_files:
    fpath = os.path.join(test_dir, fname)
    label, conf = predict_quality(fpath, model)
    print(f"{fname}: {label} ({conf:.1%} confidence)")

---

## Next Steps

1. Download the `.h5` and `.tflite` models
2. Place them in the project for Stage 2 integration
3. If accuracy < 80%, try:
   - Adding more labeled data (especially for the weaker class)
   - Increasing augmentation intensity
   - Training for more epochs
4. **Model outputs for each photo:**
   - **good** → accept the photo
   - **bad** → prompt tech to retake
   - **unreadable** → flag for manual entry (label is physically damaged)